<a href="https://colab.research.google.com/github/Aye-Nyein-Thaw/TensorFlow-Beginner/blob/main/coding-exercise/week%205/part2/3_Feature_Extraction_Layer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

## Sequential Model

In [ ]:
model = Sequential([
    Dense(6, name = 'Dense_0', input_shape = (4,)),
    Dense(4, name = 'Dense_1'),
    Dense(3, name = 'output_layer')
])

model.summary()

## Freeze second layer

In [ ]:
model.layers[1].trainable = False
model.summary()

## Load TensorFlow Hub Feature Vector

In [ ]:
import tensorflow_hub as hub

handle_base = "mobilenet_v2"
pixels = 224
output_features = 1280

IMAGE_SIZE = (pixels, pixels)
MODULE_HANDLE ="https://tfhub.dev/google/tf2-preview/{}/feature_vector/4".format(handle_base)

# The output of the module is a batch of feature vectors. 
# For each input image, the feature vector has size num_features = 1280

## Feature extraction layer

In [ ]:
feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_SIZE + (3,))

# To fine-tune this feature_extractor layer
# set feature_extractor.trainable = True

## Create Model

In [ ]:
model = Sequential([
            feature_extractor,
            Dense(2, activation='softmax')
])

## Prepare Data

In [ ]:
splits = ['train[:80%]', 'train[80%:90%]', 'train[90%:]']

splits, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=splits)

(train_examples, validation_examples, test_examples) = splits

In [ ]:
num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

class_names = ['cat', 'dog']

def format_image(image, label):
    image = tf.image.resize(image, IMAGE_SIZE) / 255.0
    return  image, label
    
BATCH_SIZE =  32

train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(BATCH_SIZE)

## Compile and train

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

history = model.fit(train_batches,
                    epochs=3,
                    validation_data=validation_batches)

## Test with our own image

In [ ]:
import numpy as np

from tensorflow.keras.preprocessing.image import load_img, img_to_array

filepath = # path to your image file, example - 'image/cat.jpg'
test_img = load_img(filepath , target_size = (pixels, pixels))

# convert image to array
img_arr = img_to_array(test_img)

# normalize
img_arr = img_arr / 255.0

# expand_dimensions
img_arr = img_arr[np.newaxis, ...]
results = model.predict(img_arr)

prediction = class_names[np.argmax(results)]
print(prediction)

display(test_img)